<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP
Merge the bibliographic datasets for Web of Scienca, Scielo and Scopus of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

In [9]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
ls WOSplus/

In [0]:
%%bash
if [ -d "drive/My Drive/WOSplus" ];then
    rm -rf "drive/My Drive/WOSplus"
    if [ ! -d "WOSplus" ];then
        git clone https://github.com/restrepo/wosplus.git
    fi
fi

In [13]:
pwd

'/content/WOSplus/wosplus'

In [0]:
import wosplus as wp
import pandas as pd

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [8]:
cat wosplus.py

#!/usr/bin/env python3
import re
import pandas as pd
from configparser import ConfigParser
try:
    from ._google_drive_tools import *
    from ._pajek_tools import *
    from ._wos_scp import *
    from ._merge_tools import *
    from ._wos_parser import *
except (SystemError, ImportError):
    from _google_drive_tools import *
    from _pajek_tools import *
    from _wos_scp import *
    from _merge_tools import *
    from _wos_parser import *

#TODO: change Tipo for Type or something similar
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_colwidth',1000)
def grep(pattern,multilinestring):
    '''Grep replacement in python
    as in: $ echo $multilinestring | grep pattern
    dev: re.M is for multiline strings
    '''
    grp=re.finditer('(.*)%s(.*)' %pattern, multilinestring,re.M)
    return '\n'.join([g.group(0) for g in grp])

def merge_inner_interior_exterior(LEFT,RIGHT,on_condition='SCP_DOI',left_on='ST',right_on='Si

In [10]:
%%writefile drive.cfg
[FILES]
UDEA_WOS.xlsx       = 1px2IcrjCrkyu7t78Q7PAE5nzV_yuPt9t
UDEA_SCI.xlsx       = 1pWMY5P72j0Ca6D-cm7dn7Q4TBGTs4PWV
UDEA_SCP.xlsx       = 1ulCsFHzDiTmuL9TH8F58ulh0u8Z2ylKh
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj

Overwriting drive.cfg


##  Load data bases

In [12]:
UDEA=wp.wosplus('drive.cfg')
tmp=UDEA.load_biblio('UDEA_WOS.xlsx') # creates WOS.WOS

ValueError: ignored

In [9]:
UDEA=wp.wosplus('drive.cfg')
tmp=UDEA.load_biblio('UDEA_WOS.xlsx') # creates WOS.WOS
tmp=UDEA.load_biblio('UDEA_SCI.xlsx',prefix='SCI') # creates UDEA.SCI
tmp=UDEA.load_biblio('UDEA_SCP.xlsx',prefix='SCP') # creates UDEA.SCP

/usr/local/lib/python3.6/dist-packages/wosplus/_wos_scp.py:65: FutureWarning: Using 'rename_axis' to alter labels is deprecated. Use '.rename' instead
  for key in df.columns.values), axis=1)


In [0]:
print('before merge: {}'.format( UDEA.WOS.shape[0]+UDEA.SCI.shape[0]+UDEA.SCP.shape[0] )  )


before merge: 24676


### WOS+SCI

In [0]:
UDEA.merge(left='WOS',right='SCI')

................................................................................

### WOS+SCI+SCP

In [0]:
UDEA.merge(left='WOS_SCI',right='SCP')

................................................................................................................................................................................................................

In [0]:
print( 'after merge: {}'.format(UDEA.WOS_SCI_SCP.shape[0]))

after merge: 15504


In [0]:
UDEA.WOS_SCI_SCP.to_excel('UDEA_WOS_SCI_SCP.xlsx',index=False)

See the resulting Excel file by opening the side bar → Files → REFRESH and right click for Download upon the generated  "'UDEA_WOS_SCI_SCP.xlsx'" fille, or just check the corresponsing DataFrame:

In [0]:
UDEA.WOS_SCI_SCP.sample()

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,SU,TC,TI,Tipo,U1,U2,UT,VL,WC,Z9
568,,"Vasquez, E.\nFranco, J. L.\nOrrego, J. C.\n",,"Vasquez, E\nFranco, JL\nOrrego, JC\n",,,,,S126\n,"[Vasquez, E.; Franco, J. L.; Orrego, J. C.] Un...",...,3\n,0.0,Subcutaneous (SCIg) Gammaglobulin for the Trea...,WOS,0.0,0.0,WOS:000347388100064\n,33\n,Immunology\n,0.0


## Load merged database

In [0]:
udea=wp.wosplus('drive.cfg')

In [0]:
#udea.load_biblio('UDEA_WOS_SCI_SCP.xlsx')

In [0]:
UDEA=udea.read_drive_excel('UDEA_WOS_SCI_SCP.xlsx')

In [0]:
UDEA=UDEA.fillna('')

In [0]:
wp.df_split( UDEA, 'DI',Operator='!=',condition='')[0].shape

(7993, 154)

In [0]:
for t in ['WOS','SCI','SCP']:
    print( 'DOIS {}: {}'.format(t,
        wp.df_split( UDEA[ UDEA.Tipo.str.contains(t) ], 'DI',Operator='!=',condition='')[0].shape) )

DOIS WOS: (6025, 154)
DOIS SCI: (1409, 154)
DOIS SCP: (6955, 154)


In [0]:
y.shape

(6025, 154)